In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize the prices
def visualize (predicted, actual, crypto):
    plt.plot(actual, color='black', label='Actual Prices')
    plt.plot(predicted, color='red', label='Predicted Prices')
    plt.title(f'{crypto} Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend(loc='upper left')
    plt.show()  

In [2]:
import pandas as pd
import pandas_datareader as web
import datetime as dt
from preprocess import scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
def scaling(data, prediction_days = 60):
    # Scaling
    
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
    
    x_train, y_train = [], []
    
    for x in range(prediction_days, len(scaled_data)):
        x_train.append(scaled_data[x-prediction_days : x, 0])
        y_train.append(scaled_data[x, 0])
        
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train

In [3]:
def predict(model, num_days, inputs):
    x_test = []
    
    for x in range(num_days, len(inputs)):
        x_test.append(inputs[x - num_days : x, 0])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    prediction_prices = model.predict(x_test)
    prediction_prices = scaler.inverse_transform(prediction_prices)

  # Predict next day
    real_data = [model_inputs[len(model_inputs) + 1 - prediction_days : len(model_inputs) + 1, 0]]
    real_data = np.array(real_data)
    real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

    prediction = model.predict(real_data)
    prediction = scaler.inverse_transform(prediction)
    print('Tomorrow\'s price will be: $', round(prediction[0][0], 2))

    return prediction_prices

In [4]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

# Neural Network
def lstm(train_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(train_shape[1], 1)))
    model.add(Dropout(0.5))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(units=25))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))
    return model

In [7]:
x_train.shape

(2634, 60, 1)

In [9]:
from model import lstm
#from predict import predict
from utils import visualize

crypto_currency = 'BTC'
against_currency = 'USD'

start = dt.datetime(2010, 1, 1)
end = dt.datetime.now()
data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', start, end)

x_train, y_train = scaling(data)

model = lstm(x_train.shape)

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=1, batch_size=32)

# Testing the model
test_start = dt.datetime(2010, 1, 1)
test_end = dt.datetime.now()

test_data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)


model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)

# Make prediction
prediction_prices = predict(model, prediction_days, model_inputs)

visualize(prediction_prices, actual_prices, crypto_currency)

83/83 [==============================] - 18s 126ms/step - loss: 0.0104


ValueError: in user code:

    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Vineet\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_